In [1]:
import re
import time
import requests
import json
import pickle
import openai

In [2]:
# OpenAI API credentials
AZURE_OPENAI_KEY = "41d471bedea9412e9d0a5461cf69bfc8"
AZURE_OPENAI_ENDPOINT = "https://openai-resource-team-6-france.openai.azure.com/"
AZURE_ENGINE_NAME = "gpt35-team-6"

openai.api_type = "azure"
openai.api_base = AZURE_OPENAI_ENDPOINT
openai.api_version = "2023-05-15"
openai.api_key = AZURE_OPENAI_KEY

In [3]:
def ask_gpt(command, text, temp = 0):
    '''
    Ask the model to generate text based on the command and the text provided.

    Parameters:
        command (str): The command to be used.
        text (str): The text to be used.
    '''
    time.sleep(1) # to avoid Azure API throttling
    content = openai.ChatCompletion.create(
        engine=AZURE_ENGINE_NAME,
        messages=[
            {"role": "system", "content": command},
            {"role": "user", "content": text}
        ],
        temperature=temp,
    )
    return content.choices[0].message["content"]

def get_story(request):
    '''
    Get a story based on the request.
    
    Parameters:
        request (str): The request from the user.
    '''
    command = "You are a helpful storyteller for children."
    return ask_gpt(command, request)

def get_summary(story):
    '''
    Get a summary of the story.

    Parameters:
        story (str): The story to be summarized.
    '''
    command = "You will be provided with a story, and your task is to summarize the story using at most 70 words."
    return ask_gpt(command, story)

def get_prompt_to_generate_image(story):
    '''
    Get a prompt to generate an image based on the story.

    Parameters:
        story (str): The story to be used.
    '''
    command = "Can you give me prompts to generate images based on this story in stable diffusion?"
    return ask_gpt(command, story)

def get_title(story, attempt=0):
    command = "You will be provided with a story, and your task is to create a title for it using at most 15 words."
    title = ask_gpt(command, story)
    num_words = len(title.split())
    if (num_words > 15 or num_words==0):
        if attempt < 3:
            return get_title(story, attempt+1)
        else:
            return "Once upon a time..."
    return title

def get_feeling(story, attempt=0):
    command = "You will be provided with a short story. Your task is to summarize the story using one word that describes the overall emotion " +\
        "of the paragraph. For example, if the story is about a happy event, you might use the word joy to summarize it. You should use one of " +\
        "the following words: happiness, sadness, anger, fear, surprise, disgust, contempt, resilience."
    feeling = ask_gpt(command, story).lower()
    if feeling not in ["happiness", "sadness", "anger", "fear", "surprise", "disgust", "contempt", "resilience", "neutral"]:
        if attempt < 3:
            return get_feeling(story, attempt+1)
        else:
            return "neutral"
    return feeling

def get_sentiment_level(story, attempt=0):
    command = "You will be provided with a short story. Your task is to get the sentiment level of the overall paragraph. " +\
        "You should use one of the following words: very positive, positive, neutral, negative, very negative."
    sentiment = ask_gpt(command, story).lower()
    if sentiment not in ["very positive", "positive", "neutral", "negative", "very negative"]:
        if attempt < 3:
            return get_sentiment_level(story, attempt+1)
        else:
            return "neutral"
    return sentiment

def get_tags(story, attempt=0):
    command = "You will be provided with a short story. Your task is to get the categories of the overall paragraph. You should use three of" +\
        " the following words: action, adventure, animation, biography, comedy, crime, documentary, drama, family, fantasy, history," +\
        " horror, music, musical, mystery, romance, sci-fi, short, sport, thriller, war, western."
    tags = ask_gpt(command, story).lower()
    tags = tags.split(", ")

    valid_tags = ["action", "adventure", "animation", "biography", "comedy", "crime", "documentary", "drama", "family", "fantasy", "history",
        "horror", "music", "musical", "mystery", "romance", "sci-fi", "short", "sport", "thriller", "war", "western"]

    # verify if list contains only 3 elements, and if each element is a valid tag
    if len(tags) != 3 or not all(elem in valid_tags for elem in tags):
        if attempt < 3:
            return get_tags(story, attempt+1)
        else:
            return ["family", "short", "animation"]
    return tags

def get_paragraphs(story):
    # split output_chatgpt into paragraphs
    paragraphs = story.split('\n\n')
    paragraphs = [x for x in paragraphs if x != '']
    # remove spaces from the beginning of each paragraph
    paragraphs = [x[1:] if x[0] == ' ' else x for x in paragraphs]
    return paragraphs

def get_scenes(story):
    scenes = []

    paragraphs = get_paragraphs(story)

    for idx, paragraph in enumerate(paragraphs):
        scene = {
            "id": str(idx),
            "text": paragraph,
            "sentiment": get_sentiment_level(paragraph),
            "polarity": get_feeling(paragraph),
            "image_prompt": get_prompt_to_generate_image(paragraph),
            "tone_song": None,
            "image": None
        }

        scenes.append(scene)

    return scenes

def create_story(request, story=None):
    '''
    The main function to create a story.

    Parameters:
        request (str): The request from the user.
        story (str): The story to be used. If None, a story will be generated.
    '''
    start = time.time()

    # For this case, the main_id is the timestamp when the story is created
    main_id = str(int(time.time()))

    if story is None:
        story = get_story(request)

    summ = get_summary(story)
    title = get_title(story)

    main_story = {
        "id": f"{main_id}_{title.replace(' ', '_')}",
        "title": title,
        "summary": summ,
        "main_prompt": request,
        "tone": get_feeling(story),
        "tags": get_tags(story),
        "cover_image_prompt": get_prompt_to_generate_image(summ),
        "story": get_scenes(story),
    }

    end = time.time()

    main_story["elapsed_time"] = end - start
    return main_story

def save_story(main):
    '''
    Save the story to a binary file (pickle) and a json file.
    '''
    
    # convert to pickle
    main_id = main["id"]
    with open(f'../../stories/{main_id}.pkl', 'wb') as f:
        pickle.dump(main, f)

    # convert to json
    main_json = json.dumps(main)
    # save to file
    with open(f'../../stories/{main_id}.json', 'w') as f:
        f.write(main_json)

In [4]:
story = "Once upon a time, there was a little boy named Timmy. Timmy was a happy and a playful boy who loved going out and playing with his friends. However, Timmy had some allergies which made it a little difficult for him to enjoy certain things like his friends did.\n\nOne day, Timmy went on a picnic with his friends. His mother had carefully packed him a lunchbox full of delicious food that she knew he could eat without any problems. As the children were playing and enjoying their food, Timmy noticed that his nose started to itch, and his eyes began to water. Suddenly, he started to feel unwell, and his throat began to hurt.\n\nTimmy's mother was nearby and noticed he wasn't feeling well. She quickly realized that Timmy was having an allergic reaction to something he had eaten. She gave him some medicine and rushed him to the hospital. Luckily, they were able to treat him before things got worse.\n\nAfterward, Timmy's mother explained to him about allergies. She told him that sometimes our bodies don't like certain things, and when we come into contact with them, it can cause us to feel unwell. She told him that it was important to always be careful with what he ate and to let others know about his allergies so they could help keep him safe.\n\nFrom that day on, Timmy was very careful about what he ate and always let his friends and family know about his allergies. He knew that being cautious was better than risking another scary reaction. With his mother's help and the knowledge she provided him, Timmy was able to have more fun and enjoy being with his friends without worrying about his allergies."

In [5]:
#story = '''Once upon a time, in a cozy little town called Sunnyville, there lived a young girl named Lily. Lily loved playing outside, chasing butterflies, and exploring the colorful meadows. But there was one thing that made Lily\'s adventures a little tricky – she had allergies.\n\nAllergies are like mischievous little creatures that live inside our bodies. Lily\'s body had some special creatures called "allergens" that made her feel sick. These allergens could be found in things like pollen from flowers, fur from cuddly animals, or even certain foods.\n\nOne sunny morning, Lily woke up with a stuffy nose and itchy, watery eyes. She knew her allergies were acting up again. Her mom noticed and said, "Oh, dear, it seems like your allergies are bothering you today. Let\'s figure out what\'s causing it."\n\nLily and her mom decided to go on an adventure together to find the culprit of her allergies. They walked through the flower-filled meadow, where Lily would often play. Suddenly, Lily sneezed! "Achoo!" It was the flowers! Lily\'s body didn\'t get along well with the pollen they released into the air.\n\nAs they continued their adventure, they met a friendly dog named Max. He wagged his tail happily, excited to meet Lily. But as Lily got closer to Max, she started to itch all over. It was Max\'s fur! Lily\'s body didn\'t like the tiny particles that clung to the dog\'s fur.\n\nLily\'s mom explained that allergies were like Lily\'s body having very picky taste buds. Just as some people might not like the taste of certain foods, Lily\'s body didn\'t like the pollen or Max\'s fur. Even though these things might be harmless to others, Lily\'s body saw them as intruders it needed to protect her from.\n\nBut Lily wasn\'t discouraged. She learned that she could still have fantastic adventures without letting her allergies ruin the fun. She started wearing a cool hat and sunglasses to shield her eyes from pollen. And whenever she encountered animals like Max, she made sure to wash her hands and clothes afterward to get rid of any allergens.\n\nLily also discovered that some foods made her feel itchy and uncomfortable. She learned to read food labels and avoid those ingredients. But there were still plenty of delicious meals and snacks she could enjoy without any problems.\n\nAs Lily embraced these new strategies, she found she could still play outside, meet new friends, and have amazing adventures. Her allergies didn\'t hold her back anymore. And whenever she needed a break, she would curl up with a good book and travel to imaginary worlds where allergies didn\'t exist.\n\nThe end.'''

In [6]:
request = "Can you explain allergies to me while telling me a children's story?"
main = create_story(request)
save_story(main)
main

{'id': '1689889707_The_Allergy_Detectives:_Spreading_Kindness_and_Knowledge',
 'title': 'The Allergy Detectives: Spreading Kindness and Knowledge',
 'summary': "Lily and her bunny friend Benny discover allergies during a picnic. They learn about the immune system's reaction to certain substances and decide to spread awareness in their village. They organize an event that promotes understanding and support for people with allergies, making their community more inclusive. Lily and Benny continue their adventures, spreading kindness and knowledge to make the world a better place.",
 'main_prompt': "Can you explain allergies to me while telling me a children's story?",
 'tone': 'resilience',
 'tags': ['family', 'adventure', 'animation'],
 'cover_image_prompt': "1. Lily and Benny sitting on a checkered picnic blanket, surrounded by colorful flowers and a wicker basket filled with sandwiches and fruits. Lily sneezes while Benny looks concerned.\n\n2. Lily and Benny standing in front of a gia

In [7]:
request = "Dermatite atópica: como explicar às crianças?"
main = create_story(request)
save_story(main)
main

{'id': '1689889970_A_Pele_Sensível_de_Ana:_Aprendendo_a_Cuidar_e_se_Divertir',
 'title': 'A Pele Sensível de Ana: Aprendendo a Cuidar e se Divertir',
 'summary': 'Ana, a girl with sensitive skin due to a condition called atopic dermatitis, teaches her friends the importance of self-care and acceptance. Despite the challenges, Ana shows that it is possible to be happy and have fun. She encourages her friends to find ways to play together without irritating her skin, emphasizing that everyone is special, regardless of their differences.',
 'main_prompt': 'Dermatite atópica: como explicar às crianças?',
 'tone': 'resilience',
 'tags': ['family', 'drama', 'short'],
 'cover_image_prompt': "1. Ana and her friends gather in a beautiful garden, surrounded by vibrant flowers and lush greenery. Ana, wearing a wide-brimmed hat and long sleeves, smiles as she shows her friends how to make natural, soothing skincare products from the flowers around them.\n\n2. Ana and her friends sit in a cozy livi

In [8]:
request = "Kevin bump is head, and he is sad."
main = create_story(request)
save_story(main)
main

{'id': '1689890259_The_Curious_Boy_and_the_Healing_Story',
 'title': 'The Curious Boy and the Healing Story',
 'summary': "Kevin, a curious and adventurous boy, bumps his head while playing in his backyard. His friend Lily comforts him with a story about a bird named Oliver who couldn't fly but found joy in other things. Inspired, Kevin realizes he can find happiness in other ways and continues his adventures with Lily.",
 'main_prompt': 'Kevin bump is head, and he is sad.',
 'tone': 'resilience',
 'tags': ['adventure', 'family', 'fantasy'],
 'cover_image_prompt': "1. Kevin and Lily sitting under a tall oak tree, with Kevin holding an ice pack to his head and Lily telling the story of Oliver the bird.\n2. A close-up of Kevin's face, his eyes wide with curiosity and a small trickle of blood on his forehead.\n3. Lily, with a gentle smile, comforting Kevin as she wraps her arm around his shoulder.\n4. A vibrant and colorful bird named Oliver, perched on a branch, looking up at the sky wit

In [9]:
request = "Mommy is having another baby."
main = create_story(request)
save_story(main)
main

{'id': '1689890550_The_Joy_of_Sibling_Love',
 'title': 'The Joy of Sibling Love',
 'summary': "Lily's family shares the exciting news that she will be a big sister. She eagerly awaits the arrival of her new sibling, helping her parents prepare for the baby's arrival. When the day finally comes, Lily meets her baby brother, Ethan, and they become the best of friends. The family of three becomes a family of four, filled with love and joy.",
 'main_prompt': 'Mommy is having another baby.',
 'tone': 'happiness',
 'tags': ['family', 'drama', 'adventure'],
 'cover_image_prompt': '1. Lily stands in her family\'s living room, a wide smile on her face, as her parents hold up a tiny onesie that reads "Big Sister" in bold letters. The room is filled with colorful balloons and streamers, creating an atmosphere of excitement and anticipation.\n\n2. Lily sits on the edge of her parents\' bed, surrounded by baby blankets and stuffed animals. She carefully folds each blanket, her small hands working w

In [10]:
request = "Why does my tummy hurt?"
main = create_story(request)
save_story(main)
main

In [ ]:
request = "What is an MRI?"
main = create_story(request)
save_story(main)
main